In [1]:
%load_ext autoreload
%autoreload 2
import random
import yaml
import os
import logging
import matplotlib.pyplot as plt
import cv2

import scipy.io as io
import torch
import torch.optim
import torch.utils.data
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import open3d as o3d
import torch.optim as optim

from tensorboardX import SummaryWriter
from utils.utils import getWriterPath
from settings import EXPER_PATH
from utils.loader import dataLoader, modelLoader, pretrainedLoader
from utils.logging import *
from copy import deepcopy as dc
from utils.d2s import DepthToSpace, SpaceToDepth
from train_cubemap import *
from utils.utils import flattenDetection
from Train_model_frontend_cubemap import thd_img
from utils_custom import *
from PIL import Image
import matplotlib.pyplot as plt

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
def measure_t(start, before):
    now = time.time()
    runt = now-start
    h = runt//3600
    m = (runt - h*3600)//60
    s = round(runt - h*3600 - m*60)
    prtstr = f"iter {_iter+1} in {len(train_loader)}, \
            {round((_iter+1)/len(train_loader), 4) * 100}%\
            {h}h {m}m {s}s"
    runt2 = before-start
    h2 = runt2//3600
    m2 = (runt2 - h2*3600)//60
    s2 = round(runt2 - h2*3600 - m2*60)
    prtstr += f"  ({h2}h {m2}m {s2}s)"
    return now  
args = Namespace(command='train_joint', config='configs/magicpoint_cubemap.yaml', debug=False, eval=False, exper_name='cubemap_dataset', func=train_joint)


from utils_custom import *
from utils_custom_visualize import *

with open(args.config, 'r') as f:
    config = yaml.safe_load(f)
EXPER_PATH = 'logs'
output_dir = EXPER_PATH
output_dir

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'logs'

In [2]:
############################################### train_joint

torch.set_default_tensor_type(torch.FloatTensor)
task = config['data']['dataset']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info('train on device: %s', device)
with open(os.path.join(output_dir, 'config.yml'), 'w') as f:
    yaml.dump(config, f, default_flow_style=False)
# writer = SummaryWriter(getWriterPath(task=args.command, date=True))
writer = SummaryWriter(getWriterPath(task=args.command, 
    exper_name=args.exper_name, date=True))
## save data
save_path = get_save_path(output_dir)

############################################### 
data = dataLoader(config, dataset=task, warp_input=True)
train_loader, val_loader = data['train_loader'], data['val_loader']

datasize(train_loader, config, tag='train')
datasize(val_loader, config, tag='val')
# init the training agent using config file
# from train_model_frontend import Train_model_frontend
from utils.loader import get_module
train_model_frontend = get_module('', config['front_end_model'])
train_agent = train_model_frontend(config, save_path=save_path, device=device)

# writer from tensorboard
train_agent.writer = writer

# feed the data into the agent
train_agent.train_loader = train_loader
train_agent.val_loader = val_loader

# load model initiates the model and load the pretrained model (if any)
train_agent.loadModel()

2023-05-04 09:34:42 koala-A520M-H root[4012480] INFO train on device: cuda
2023-05-04 09:34:42 koala-A520M-H root[4012480] INFO => will save everything to logs/checkpoints
2023-05-04 09:34:42 koala-A520M-H root[4012480] INFO workers_train: 1, workers_val: 1


dataset: CubemapDataset
not_warped_images:  False
not_warped_images:  False


2023-05-04 09:34:43 koala-A520M-H root[4012480] INFO == train split size 64800 in 64800 batches
2023-05-04 09:34:43 koala-A520M-H root[4012480] INFO == val split size 64800 in 64800 batches
2023-05-04 09:34:43 koala-A520M-H root[4012480] INFO => creating model: SuperPointNet_cubemap


HOMONUM: 100   HOMOBATCH: 1
use sparse_loss!
set train loader
set train loader
model:  SuperPointNet_cubemap


2023-05-04 09:34:44 koala-A520M-H root[4012480] INFO => setting adam solver
2023-05-04 09:34:44 koala-A520M-H root[4012480] INFO reset iterations to 0


load pretrained model from: %s logs/iter333_ce/checkpoints/RD_230504_091857_iter334_checkpoint.pth.tar
successfully load pretrained model from: %s logs/iter333_ce/checkpoints/RD_230504_091857_iter334_checkpoint.pth.tar


 Train only descriptor


In [3]:
sample =next(iter(train_loader))
net = dc(train_agent.net)
optimizer = optim.Adam(net.parameters(), lr=0.1, betas=(0.9, 0.999))
print(sample.keys(), sample['image'].shape)
img, img_w = sample['image'].to(device), sample['warped_image'].to(device)
batch_size, H, W = img.shape[0], img.shape[2], img.shape[3]

dict_keys(['image', 'warped_image', 'ply_path', 'R', 'T', 'R_w', 'T_w', 'img_path', 'img_path_w', 'kpts2D', 'kpts3D', 'kpts2D_w', 'kpts3D_w']) torch.Size([1, 3, 1024, 1024])


In [4]:
sample.keys(), sample['image'].shape

(dict_keys(['image', 'warped_image', 'ply_path', 'R', 'T', 'R_w', 'T_w', 'img_path', 'img_path_w', 'kpts2D', 'kpts3D', 'kpts2D_w', 'kpts3D_w']),
 torch.Size([1, 3, 1024, 1024]))

In [5]:
thd = 0.05
B, h, b, loss = 0, 0, 0, 0

bh = 0
n_iter = 1  ##########


dbk2d, dbk2d_w, dbk3d, dbk3d_w = sample['kpts2D'][B],sample['kpts2D_w'][B],sample['kpts3D'][B],sample['kpts3D_w'][B]        

H_NUM_THIS = 2

im_trf_cat, Hinv_infos = get_homo_img_cat(img, H_NUM_THIS)
im_trf_cat_w, Hinv_infos_w = get_homo_img_cat(img_w, H_NUM_THIS)

outs, outs_warp = (
    net(im_trf_cat[B]),  # B는 배치를 뜻함, 배치는 1로 고정, (b, bh, c, H, W)
    net(im_trf_cat_w[B]),# 모델 인풋은 4차원이어야 하므로 bh를 b처럼 이용
    )
semi, coarse_desc = outs['semi'], outs['desc']
hms = flattenDetection(semi)
semi_w, coarse_desc_w = outs_warp['semi'], outs_warp['desc']
hms_w = flattenDetection(semi_w)

desc = train_agent.interpolate_to_dense(coarse_desc)[bh]
desc_w = train_agent.interpolate_to_dense(coarse_desc_w)[bh]


In [6]:
desc.shape

torch.Size([256, 1024, 1024])

In [7]:
from utils.loss_functions.custom_loss import *

imgshape = (H_NUM_THIS, H, W)
labels3D_in_loss = getLabels(imgshape, dbk2d, device=device)
labels3D_in_loss_w = getLabels(imgshape, dbk2d, device=device)

from copy import deepcopy as dc
############################################################################
mask_3D_flattened, mask2D = getMasks(imgshape, dc(Hinv_infos[bh]), device=device)
loss_det = detection_loss_custom(
    semi[bh], labels3D_in_loss[bh], mask_3D_flattened, device=device
)

mask_3D_flattened_w, mask2D_w = getMasks(imgshape, dc(Hinv_infos_w[bh]), device=device)
loss_det_w = detection_loss_custom(
    semi_w[bh], labels3D_in_loss_w[bh], mask_3D_flattened_w, device=device
)
############################################################################

hm = apply_H_from_info(hms[bh], Hinv_infos[bh])
hm = thd_img(hm, thd=thd)
kpts = get_kpts_from_hm(hm, mask2D)

hm_w = apply_H_from_info(hms_w[bh], Hinv_infos_w[bh])
hm_w = thd_img(hm_w, thd=thd)
kpts_w = get_kpts_from_hm(hm_w, mask2D_w)

/home/koala/anaconda3/envs/pnp/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [8]:
matched_kpts_idx, matched_kpts_idx_w = get_matches(
    dbk2d, dbk3d.tolist(), dbk2d_w, dbk3d_w.tolist(),
    kpts, kpts_w,
    device=device,
)

In [9]:
kpts_desc = get_desc_of_kpts(desc, kpts)
kpts_desc_w = get_desc_of_kpts(desc_w, kpts_w)
kpts_desc = kpts_desc.transpose(0, 1)
kpts_desc_w = kpts_desc_w.transpose(0, 1)

In [10]:
len(kpts), kpts_desc.shape, len(kpts_w), kpts_desc_w.shape

(1987, torch.Size([1987, 256]), 3797, torch.Size([3797, 256]))

In [11]:

kpts = torch.Tensor(kpts).to(device)
kpts_w = torch.Tensor(kpts_w).to(device)
print(kpts.shape, kpts_desc.shape)
plt.figure()

matched_kpts_idx, matched_kpts_idx_w = torch.Tensor(matched_kpts_idx), torch.Tensor(matched_kpts_idx_w)
maching_plot(img, img_w, kpts, kpts_w, 
    kpts[matched_kpts_idx.int()], kpts_w[matched_kpts_idx_w.int()], 
    path=os.path.join(f'test_training_this.png'))
plt.close()                
kpts_sorted, kpts_w_sorted, color = sort_by_similar_pts(kpts, kpts_w, kpts_desc, kpts_desc_w)

torch.Size([1987, 2]) torch.Size([1987, 256])


In [12]:
kpts_desc = get_desc_of_kpts(desc, kpts)
kpts_desc_w = get_desc_of_kpts(desc_w, kpts_w)
kpts_desc = kpts_desc.transpose(0, 1).detach().cpu()
kpts_desc_w = kpts_desc_w.transpose(0, 1).detach().cpu()
kpts_sorted, kpts_w_sorted, color = sort_by_similar_pts(kpts.int(), kpts_w.int(), kpts_desc, kpts_desc_w)
plt.figure()
maching_plot(img, img_w, kpts, kpts_w, 
    kpts_sorted.detach().cpu(), kpts_w_sorted.detach().cpu(), color,
    path=os.path.join(f'test_predicted_this.png'))
plt.close()

In [13]:
outfig = maching_plot(sample['img_path'][0], sample['img_path_w'][0], 
        kpts, kpts_w, 
        kpts[matched_kpts_idx], kpts_w[matched_kpts_idx_w],
                                 img_shape=img.shape[2:])
plt.imshow(outfig)
plt.savefig('rd_glue0.jpg', dpi=1200)

IndexError: tensors used as indices must be long, int, byte or bool tensors

In [ ]:
len(matched_kpts_idx)

In [ ]:
import torch
import numpy as np
# a = torch.Tensor([[1,1],[2,2],[3,3],[0,0],[0,0], [5,0]])
idx = torch.where(a[:,1]+a[:,0] != 0)[0]
a[idx]

In [ ]:
a = np.array([[1,1],[2,2],[3,3],[0,0],[0,0], [5,0]])
idx = np.where(a[:,1]+a[:,0] != 0)[0]
a[idx].tolist()